In [1]:
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_excel("Online Retail.xlsx")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#### Action Point

RFM Metrics

Calculate for recency, frequency and monetary 

1. investigate missing numbers 
2. drop irrelevant columns 
3. calculate for RFM
  I.   Invoice date = recency
  II.  Count of InvoiceNo = frequency
  III. Undefined (SUM: Quantity x UnitPrice) = Monetary
  IV.  Create RFM score
4. Bin RFM score and segment customer
5. insight and recommendation 

In [4]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
df.Country.nunique()

38

In [6]:
df.Country.value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [7]:
#Query UK alone for the analyis
df = df[df['Country'] == 'United Kingdom']

In [8]:
df.Country.unique()

array(['United Kingdom'], dtype=object)

In [9]:
#to drop irrelevant columns starting with description.

df.drop(['Description','Country'], axis=1, inplace=True)

In [10]:
#confirming code
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0


In [11]:
#calculate for monetary
df['Monetary'] = df['Quantity']*df['UnitPrice']

In [12]:
#confirming code
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Monetary
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,15.30
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,20.34
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,22.00
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,20.34
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,20.34


In [13]:
 # We assume that we are doing this analysis 1 day after from latest transaction on the data.

Todays_date = max(df.InvoiceDate) + dt.timedelta(days = 1)

In [14]:
Todays_date

Timestamp('2011-12-10 12:49:00')

In [19]:
#lambda 

data = df.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (Todays_date - x.max()).days,'InvoiceNo': 'count','Monetary': 'sum'})

data.rename(columns={'InvoiceDate': 'Recency','InvoiceNo': 'Frequency'}, inplace=True )

In [21]:
data.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12747.0,2,103,4196.01
12748.0,1,4642,29072.10
12749.0,4,231,3868.20
12820.0,3,59,942.34


In [23]:
data.to_csv('RFM_Data.csv')